In [19]:
import torch
import pandas as pd
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from data_treatment import DataSet, DataAtts
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import glob

In [20]:
#Run train_generator_discriminator.py before running this

folder = widgets.Dropdown(
    options=glob.glob("models/*"),
    description='Folder:',
    value="models/diabetes_escalonated",
    disabled=False,
)
display(folder)

Dropdown(description='Folder:', options=('models/diabetes_escalonated',), value='models/diabetes_escalonated')

In [21]:
folder_name = folder.value+"/generator*.pt"
model_widget = widgets.Dropdown(
    options=glob.glob(folder_name),
    description='Generator:',
    disabled=False,
)
display(model_widget)

Dropdown(description='Generator:', options=('models/diabetes_escalonated/generator_id-2_epochs-500_layer-2_lr-…

In [22]:
original_db_name = folder.value[7:]
original_db_path = "original_data/" + original_db_name + ".csv"
original_db = pd.read_csv(original_db_path)
original_db_size=original_db.shape[0]

In [23]:
try:
    checkpoint= torch.load(model_widget.value, map_location='cuda')
except:
    checkpoint= torch.load(model_widget.value, map_location='cpu')
print(model_widget.value)
checkpoint['model_attributes']['out_features'] = len(original_db.columns)
generator = GeneratorNet(**checkpoint['model_attributes'])
generator.load_state_dict(checkpoint['model_state_dict'])

models/diabetes_escalonated/generator_id-2_epochs-500_layer-2_lr-0.0002_batch-5_arc-256,512.pt


<All keys matched successfully>

In [24]:
size = original_db_size
new_data = generator.create_data(size)
df = pd.DataFrame(new_data, columns=original_db.columns)
#Changes the name to be easier to read
name = model_widget.value.split("/")[-1][10:-4] + "_size-" + str(size)
df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
#Do the same thing as the cells above but for all the files in the directory
import glob
for file in glob.glob(folder_name):
    name = file.split("/")[-1][10:-4]
    print(name)
    try:
        checkpoint= torch.load(file, map_location='cuda')
    except:
        checkpoint= torch.load(file, map_location='cpu')
    generator = GeneratorNet(**checkpoint['model_attributes'])
    generator.load_state_dict(checkpoint['model_state_dict'])
    size = original_db_size
    new_data = generator.create_data(size)
    df = pd.DataFrame(new_data, columns=original_db.columns)
    name = name + "_size-" + str(size)
    df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)